# IEDB Query API (IQ-API) - Use Case 1A
**Goal**: Search for information related to a specific linear epitope, using 'SIINFEKL' as an example.

This document illustrates some basic usage of the IEDB query API.  It is by no means meant to be comprehensive or authoritative, as many tasks can be accomplished in multiple different ways.  Here we focus on simple queries of individual tables.  For more information on the expressive syntax of PostgresT, refer to [this document](https://postgrest.org/en/stable/api.html#).  For more details on the tables that are part of the API, refer to [the swagger documetation](http://query-api.iedb.org/docs/swagger/).

Some of the queries in this example will take longer than others...be patient.  Also please keep in mind that the database is rebuilt weekly so results may change from one run of this document to the next.  Along those lines, also note that this is an early beta so it is possible that some of the table or column names may change prior to the production release.

With all that in mind, let's have some fun!

---

First, let's import required modules, set some globals, and define a function to print the corresponding CURL command for each request.  I've tried to include that CURL command for each example so that you can copy/paste it into your terminal.  You may want to pipe the output to a tool like 'jq' to have it render neatly.

In [1]:
import requests
import json
import pandas as pd
from io import StringIO

base_uri='https://query-api.iedb.org'

# funciton to print the CURL command given a request
def print_curl_cmd(req):
    url = req.url
    print("curl -X 'GET' '" + url + "'")

/Users/jgbaum/.pyenv/versions/3.8.2/envs/jupyter/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


This may or may not have resulted in a warning about lzma compression.  That can be safely ignored...

### Epitope Search
Search for all epitopes with 'SIINFEKL' as the linear sequence.  We use the postgresT 'eq' operator to denote equality.

In [2]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='epitope_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)

curl -X 'GET' 'https://query-api.iedb.org/epitope_search?linear_sequence=eq.SIINFEKL'


OK we have the result...now let's have a look.  **Note**: We only print the first record that is returned here since the output can be long and confusing.  You'll see...

In [3]:
print(result.json()[:1])

[{'structure_id': 58560, 'structure_iri': 'IEDB_EPITOPE:58560', 'structure_descriptions': ['SIINFEKL'], 'curated_source_antigens': [{'accession': '0705172A', 'name': 'ovalbumin', 'iri': 'GENPEPT:0705172A', 'starting_position': 257, 'ending_position': 264, 'source_organism_name': 'Gallus gallus (chicken)', 'source_organism_iri': 'NCBITaxon:9031'}, {'accession': 'AAA48998.1', 'name': 'ovalbumin', 'iri': 'GENPEPT:AAA48998.1', 'starting_position': 258, 'ending_position': 265, 'source_organism_name': 'Gallus gallus (chicken)', 'source_organism_iri': 'NCBITaxon:9031'}, {'accession': 'P01012.2', 'name': 'Ovalbumin', 'iri': 'GENPEPT:P01012.2', 'starting_position': 258, 'ending_position': 265, 'source_organism_name': 'Gallus gallus (chicken)', 'source_organism_iri': 'NCBITaxon:9031'}], 'structure_type': 'Linear peptide', 'linear_sequence': 'SIINFEKL', 'e_modification': None, 'linear_sequence_length': 8, 'iedb_assay_ids': [24151, 24154, 203468, 203469, 1020693, 1020694, 1020695, 1020696, 1022871

OK that's hard to parse, let's have a look at a table representation instead.

In [4]:
df = pd.json_normalize(result.json())
df

,structure_id,structure_iri,structure_descriptions,curated_source_antigens,structure_type,linear_sequence,e_modification,linear_sequence_length,iedb_assay_ids,iedb_assay_iris,...,bcell_ids,bcell_iris,elution_ids,elution_iris,journal_names,reference_types,pubmed_ids,reference_titles,reference_authors,reference_dates
0,58560,IEDB_EPITOPE:58560,[SIINFEKL],"[{'accession': '0705172A', 'name': 'ovalbumin'...",Linear peptide,SIINFEKL,None,8,"[24151, 24154, 203468, 203469, 1020693, 102069...","[IEDB_ASSAY:1020693, IEDB_ASSAY:1020694, IEDB_...",...,"[1804354, 1804361, 1822199, 1957275, 3924055, ...","[IEDB_ASSAY:1804354, IEDB_ASSAY:1804361, IEDB_...","[203468, 203469, 1020693, 1020694, 1020695, 10...","[IEDB_ASSAY:1020693, IEDB_ASSAY:1020694, IEDB_...","[ACS Nano, Aging Cell, Am J Transplant, Angew ...","[Literature, Submission]","[10449165, 10477598, 10570269, 10671225, 11180...",[Absence of tapasin alters immunodominance aga...,[Aaron S Rapaport; Jill Schriewer; Susan Gilfi...,"[1992, 1993, 1994, 1995, 1996, 1997, 1998, 199..."
1,114230,IEDB_EPITOPE:114230,[SIINFEKL + SCM(K7)],"[{'accession': 'AAA48998.1', 'name': 'ovalbumi...",Linear peptide,SIINFEKL,Side chain modification,8,"[1667743, 1677301, 1678654, 1953163, 1953167, ...","[IEDB_ASSAY:1667743, IEDB_ASSAY:1677301, IEDB_...",...,None,None,"[1677301, 1953163, 1953167]","[IEDB_ASSAY:1677301, IEDB_ASSAY:1953163, IEDB_...","[Exp Dermatol, J Exp Med, J Immunol]",[Literature],"[12631250, 1383319, 8335903, 9151706]",[A high frequency of allergen-specific CD8+ Tc...,[S Martin; A von Bonin; C Fessler; U Pflugfeld...,"[1992, 1993, 1997, 2003]"
2,422946,IEDB_EPITOPE:422946,[SIINFEKL + BIOT(L8)],"[{'accession': '0705172A', 'name': 'ovalbumin'...",Linear peptide,SIINFEKL,Biotin|BIOT,8,[2468230],[IEDB_ASSAY:2468230],...,None,None,[2468230],[IEDB_ASSAY:2468230],[Eur J Immunol],[Literature],[20017190],[Tapasin edits peptides on MHC class I molecul...,[P V K Praveen; Rakina Yaneva; Hubert Kalbache...,[2010]


That matches our search on the IEDB website, where there are 3 epitope records returned.

What if we don't need all of the columns that are returned?  Maybe we only want the structure IDs and a few other fields.  We can accomplish that by passing the 'select' parameter with a list of the fields we want to retrieve.  

In [5]:
search_params={  'linear_sequence': 'eq.SIINFEKL',
                'select': 'structure_id, structure_iri, linear_sequence, tcell_ids, tcell_irs, bcell_ids, bcell_iris'}
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
print(result.json())


curl -X 'GET' 'https://query-api.iedb.org/epitope_search?linear_sequence=eq.SIINFEKL&select=structure_id%2C+structure_iri%2C+linear_sequence%2C+tcell_ids%2C+tcell_irs%2C+bcell_ids%2C+bcell_iris'
{'hint': 'Perhaps you meant to reference the column "epitope_search.tcell_ids" or the column "epitope_search.tcell_iris".', 'details': None, 'code': '42703', 'message': 'column epitope_search.tcell_irs does not exist'}


Oops, we made a spelling error.  Look at the helpful error message!  Let's try again....

In [6]:
search_params={  'linear_sequence': 'eq.SIINFEKL',
                'select': 'structure_id, structure_iri, linear_sequence, tcell_ids, tcell_iris, bcell_ids, bcell_iris'}
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

curl -X 'GET' 'https://query-api.iedb.org/epitope_search?linear_sequence=eq.SIINFEKL&select=structure_id%2C+structure_iri%2C+linear_sequence%2C+tcell_ids%2C+tcell_iris%2C+bcell_ids%2C+bcell_iris'


,structure_id,structure_iri,linear_sequence,tcell_ids,tcell_iris,bcell_ids,bcell_iris
0,58560,IEDB_EPITOPE:58560,SIINFEKL,"[24151, 24154, 1022871, 1068817, 1068827, 1273...","[IEDB_ASSAY:1022871, IEDB_ASSAY:1068817, IEDB_...","[1804354, 1804361, 1822199, 1957275, 3924055, ...","[IEDB_ASSAY:1804354, IEDB_ASSAY:1804361, IEDB_..."
1,114230,IEDB_EPITOPE:114230,SIINFEKL,"[1667743, 1678654, 1953168, 1953203]","[IEDB_ASSAY:1667743, IEDB_ASSAY:1678654, IEDB_...",None,None
2,422946,IEDB_EPITOPE:422946,SIINFEKL,None,None,None,None


Awesome.  Note the additional complexity in the URL of the last two queries.  There are two parameters (linear_sequence & select), multiple values for the latter parameter, and many URL escape codes for the commas.  Python's 'request' module handles this all for you, but one should be aware that all portions of the query need to be URL-escaped.

What if we want to search for multiple sequences?  Then we'll need to use the postgres 'in' operator in our search term.  E.g., here we search for two different sequences.

In [ ]:
search_params={  'linear_sequence': 'in.(SIINFEKL,GILGFVFTL)',
                'select': 'structure_id, structure_iri, linear_sequence, tcell_ids, tcell_iris, bcell_ids, bcell_iris'}
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

Cool.  And since we've pulled everything into a pandas dataframe, we can opt do to additional filtering here.  For instance, if we only want the epitopes that have associated B cell assays:

In [ ]:
df[df['bcell_ids'].notnull()]

### Antigen Search
Search for all antigens that are a parent protein of 'SIINFEKL'.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='antigen_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### T Cell Search
Search for all T cell assays that test the linear sequence 'SIINFEKL'. Similar to the 'epitope' search, we use the 'eq' operator.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='tcell_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### B Cell Search
Search for all B cell assays that test the linear sequence 'SIINFEKL'. Similar to the 'epitope' search, we use the 'eq' operator.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='bcell_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### MHC Search
Search for all MHC assays that test the linear sequence 'SIINFEKL'. Similar to the 'epitope' search, we use the 'eq' operator.

In [ ]:
search_params={ 'linear_sequence': 'eq.SIINFEKL'}
table_name='mhc_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### TCR Search
Search for all TCRs that recognize the linear sequence 'SIINFEKL'.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='tcr_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
df = pd.json_normalize(result.json())
df

### BCR Search
Search for all BCRs that recognize the linear sequence 'SIINFEKL'.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='bcr_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

### References
Search for all references where the linear sequence 'SIINFEKL' was tested in an assay.  Since the 'linear_sequences' field is an array of linear sequences associated with the antigen, we must use the postgres 'contains' operator, expressed as 'cs' in postgresT.

In [ ]:
search_params={ 'linear_sequences': 'cs.{SIINFEKL}'}
table_name='reference_search'
full_url=base_uri + '/' + table_name
result = requests.get(full_url, params=search_params)
print_curl_cmd(result)
df = pd.json_normalize(result.json())
df

---
This document was prepared by [Jason Greenbaum](mailto:jgbaum@lji.org) and is licensed under a [Creative Commons Attribution-NonCommercial 4.0 International License](https://creativecommons.org/licenses/by-nc/4.0/). [![](https://i.creativecommons.org/l/by-nc/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc/4.0/)